# License

In [1]:
# Copyright 2017 The TensorFlow Authors All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

def installDeps():
    !pip install numpy scipy
    !pip install resampy tensorflow six
    !pip install youtube_dl
    !pip install ipywidgets
    !pip install pydub
    !pip install tqdm
    !pip install ffmpeg-python
    !apt-get install ffmpeg -y
    #!wget https://storage.googleapis.com/audioset/vggish_model.ckpt
    #!wget https://storage.googleapis.com/audioset/vggish_pca_params.npz
#installDeps()
#!python vggish_train_demo.py --num_batches 50 --train_vggish=False --checkpoint './vggish_model.ckpt'

from __future__ import print_function

from random import shuffle

import numpy as np
import tensorflow as tf
import os
import vggish_input
import vggish_params
import vggish_slim
from pydub import AudioSegment
from audioModel import predict, train
from audioInput import getLaughTracks, getNoise

def trainAndSaveAndPredict(test_data, number_of_classes = 2, number_of_samples = 1, epochs = 5, getData = getLaughTracks, use_cache = True, log = True):
    def curriedGetSamples(shuf):
        return getData(number_of_samples = number_of_samples, shuf = shuf, use_cache = use_cache, log = log)
    preds = train(curriedGetSamples, number_of_classes, model_name = 'audio', epochs = epochs)
    

    return predict(model_name, number_of_classes, test_data)

def printResults(preds, expected = None): 
    with tf.Graph().as_default(), tf.Session() as sess:
        print(preds)
        print(sess.run(tf.argmax(input=preds, axis=1))) 
        print('expected results', expected)

def trainForNoise(number_of_samples=5, epochs=5):
    use_cache = False
    print('training on noise, sin, and constant waves')
    (features, labels) = getNoise(shuf=False, number_of_samples = 2)
    preds = trainAndSaveAndPredict(features, number_of_classes = 3, number_of_samples = number_of_samples, epochs = epochs, getData = getNoise)
    printResults(preds, [0, 0, 1, 1, 2, 2])
    
def trainForLaughter(number_of_samples=5, epochs=5):  
    use_cache = False
    #print('training on laughter and not laughter')
    (features, labels) = getLaughTracks(shuf=False, number_of_samples = 2, use_cache = use_cache, log=False)
    preds = trainAndSaveAndPredict(features, number_of_classes = 2, number_of_samples = number_of_samples, epochs = epochs, getData = getLaughTracks, use_cache = use_cache, log = False)
    #printResults(preds, [0, 0, 1, 1])


In [2]:
trainForLaughter(number_of_samples=800, epochs=100)

INFO:tensorflow:Restoring parameters from ./vggish_model.ckpt
Step 1: loss 0.723531


NameError: name 'number_of_samples' is not defined

In [ ]:
#for i in range (0, 30):
#    number_of_samples = 1006 + (i * 1)
#    trainForLaughter(number_of_samples=number_of_samples, epochs=1)

In [ ]:
trainForLaughter(number_of_samples=800, epochs=40)